In [19]:
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import scipy
import numpy as np
import os

from parse_simlib import parse_simlib
from python_utilities.des_io import parse_observations

In [98]:
#Creates a list of all pointings, with [LIBID, RA, DECL, MJD IDEXPT  FLT GAIN NOISE SKYSIG (pixels)  RATIO  ZPTAVG ZPTERR  MAG]
libid_details, pointings_arr = parse_simlib('../minion_1016_WFD.simlib')

print len(pointings_arr)


2083758


In [7]:
def load_KN_locs(DIR): #load locations and times of all DAT files for KN in given directory 
    file_count = len([f for f in os.walk(DIR).next()[2] if f[-4:] == ".DAT"])
    KN_locations = np.zeros([file_count, 3])

    file_num = 0
    for file in os.listdir(DIR):  # run through all .dat files in the specified directory
        if file.endswith(".DAT"):
            obs, headerdict = parse_observations(DIR + file)
            KN_locations[file_num,:] = [headerdict['PEAKMJD'], headerdict['RA'], headerdict['DECL']]
            file_num += 1
    return KN_locations

KN_locations = load_KN_locs('../GW170817_AT2017gfo_LSST_WFD/')

In [142]:
KN = KN_locations[0,:]

When is the KN going to be visible? Check the next 24 hours in 1 hour intervals

In [143]:
#when is the KN going to be visible? Check the next 24 hours in 1 hour intervals
lsst_location = EarthLocation(lat=-30.2446*u.deg, lon=-70.7494*u.deg, height=2663*u.m)
utcoffset = -3*u.hour  # Chile Daylight Time

times = np.arange(KN[0], KN[0]+1, 1/24.) #1 hour
time = Time(times, format='mjd') - utcoffset
KN_loc = SkyCoord(ra=KN[1] * u.degree, dec=KN[2] * u.degree)
KNaltaz = KN_loc.transform_to(AltAz(obstime=time,location=lsst_location))  
altitude = KNaltaz.alt.deg

observable = (altitude>5).astype(int) #observable if altitude in sky is >5deg
for i,t in enumerate(times):
    if 0.45 < t%1 <0.945: #must be dark. All pointings in simlib satisfy this condition
        observable[i]=0

print observable
print KN

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[  6.17753400e+04   6.90160900e+01  -5.85892330e+01]


Are we scheduled to observe this event anyway? If not find the closest observation in the next 24 hours

In [164]:
#get pointings in the next week and day
pointings_later = pointings_arr[KN[0]<pointings_arr[:,3], :]
pointings_nextweek = pointings_later[pointings_later[:,3]<(KN[0]+7)]
pointings_nextday = pointings_later[pointings_later[:,3]<(KN[0]+1)]
#print pointings_nextday[:,3]

#check if we are scheduled to observe this event anyway
if len(pointings_nextday) == 0: #if no pointings in the next 24hrs
    print 'Problem: no pointings in the next day'
else:
    #get angles between all of pointings and KN
    pointings_locs = SkyCoord(ra=pointings_nextday[:,1] * u.degree, dec=pointings_nextday[:,2] * u.degree)
    angsep = KN_loc.separation(pointings_locs).deg
    observations_arg = np.argwhere(angsep < np.sqrt(9.6/np.pi))
    KN_observations = pointings_nextday[observations_arg,:]
    if len(KN_observations) != 0:
        obs_scheduled = 1
        closest_obs = np.nan
    else: #find closest pointing within 24hours
        obs_scheduled = 0
        closest_obs = pointings_nextday[np.argmin(angsep),:]
        

print closest_obs, swap_angle

[  3.14000000e+02   4.79445350e+01  -6.20211340e+01   6.17761496e+04
   1.47971900e+06   3.00000000e+00   1.00000000e+00   2.50000000e-01
   4.32800000e+01   2.55000000e+00   0.00000000e+00   0.00000000e+00
   3.13000000e+01   5.00000000e-03  -9.90000000e+01] 10.9307324221


Search for future pointing at KN location (within 0.5deg) in next 30 days to swap the pointing in the next 24 hours with. What is the cost in slew time of swapping these pointings?

In [179]:
if obs_scheduled == 0:
    longterm_pointings_locs = SkyCoord(ra=pointings_nextmonth[:,1] * u.degree, dec=pointings_nextmonth[:,2] * u.degree)
    angsep_longterm = KN_loc.separation(longterm_pointings_locs).deg
    observations_arg = np.argwhere(angsep_longterm < 0.5)
    pointing_to_swap = pointings_nextmonth[observations_arg,:][0][0]

    #what is cost of the swap in overhead time
    swap_to_now = SkyCoord(ra=closest_obs[1] * u.degree, dec=closest_obs[2] * u.degree)
    swap_to_later = SkyCoord(ra=pointing_to_swap[1] * u.degree, dec=pointing_to_swap[2] * u.degree)
    swap_angle = swap_to_now.separation(swap_to_later).deg 
    const1, const2 = 0, 0
    new_overhead_time = const1 + const2*swap_angle
    original_overhead_time = pointings_nextday[np.argmin(angsep),3] - pointings_nextday[np.argmin(angsep)-1,3]
    extra_overhead_time = new_overhead_time - original_overhead_time #units of days
    
    print pointing_to_swap, extra_overhead_time

[  3.82000000e+02   6.90160900e+01  -5.85892320e+01   6.17771328e+04
   1.48036800e+06   3.00000000e+00   1.00000000e+00   2.50000000e-01
   4.70100000e+01   1.84000000e+00   0.00000000e+00   0.00000000e+00
   3.13400000e+01   5.00000000e-03  -9.90000000e+01] -0.000499999994645
